In [1]:


import sparknlp
 
spark = sparknlp.start() # for GPU training >> sparknlp.start(gpu = True) # for Spark 2.3 =>> sparknlp.start(spark23 = True)
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *

from sparknlp.common import *

import sparknlp
from sparknlp.pretrained import PretrainedPipeline
 
print("Spark NLP version", sparknlp.version())
 
print("Apache Spark version:", spark.version)



22/10/29 12:22:04 WARN Utils: Your hostname, Vader resolves to a loopback address: 127.0.1.1; using 192.168.1.94 instead (on interface enp5s0f0)
22/10/29 12:22:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/sjhjrok/anaconda3/envs/NLP/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/sjhjrok/.ivy2/cache
The jars for the packages stored in: /home/sjhjrok/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6a4c6a2f-008d-4089-b309-f15381e4f9be;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;4.2.2 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.code.findbugs#annotations;3.0.1 in central
	found net.jcip#jcip-annotations;1.0 in central
	found com.google.code.findbugs#jsr305;3.0.1 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#l

22/10/29 12:22:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Spark NLP version 4.2.2
Apache Spark version: 3.3.1


In [2]:


from sparknlp.training import CoNLL

training_data = CoNLL().readDataset(spark, './eng_custom.train')
testing_data= CoNLL().readDataset(spark, './eng_custom.testa')



In [3]:
import pyspark.sql.functions as F

#training_data.select(F.explode(F.arrays_zip('token.result', 'pos.result',  'label.result')).alias("cols")) \
#.select(F.expr("cols['0']").alias("token"),
#        F.expr("cols['1']").alias("pos"),
#        F.expr("cols['2']").alias("ner_label")).show(truncate=50)


In [4]:
!mkdir ner_logs

mkdir: cannot create directory ‘ner_logs’: File exists


In [5]:


# You can use any word embeddings you want (Glove, Elmo, Bert, custom etc.)
document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')
embeddings = WordEmbeddingsModel.pretrained('glove_100d')\
          .setInputCols(["document", "token"])\
          .setOutputCol("embeddings")

nerTagger = NerDLApproach()\
      .setInputCols(["sentence", "token", "embeddings"])\
      .setLabelColumn("label")\
      .setOutputCol("ner")\
      .setMaxEpochs(500)\
      .setLr(0.003)\
      .setBatchSize(32)\
      .setRandomSeed(0)\
      .setVerbose(1)\
      .setValidationSplit(0.2)\
      .setEvaluationLogExtended(True) \
      .setEnableOutputLogs(True)\
      .setIncludeConfidence(True)\
      .setOutputLogsPath('ner_logs') # if not set, logs will be written to ~/annotator_logs
 #    .setGraphFolder('graphs') >> put your graph file (pb) under this folder if you are using a custom graph generated thru 4.1 NerDL-Graph.ipynb notebook
 #    .setEnableMemoryOptimizer() >> if you have a limited memory and a large conll file, you can set this True to train batch by batch 
    
ner_converter = NerConverter() \
    .setInputCols(['document', 'token', 'ner']) \
    .setOutputCol('ner_chunk')

ner_pipeline = Pipeline(stages=[
      embeddings,
      nerTagger,
      ner_converter
 ])



glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[ | ]glove_100d download started this may take some time.
Approximate size to download 145.3 MB
Download done! Loading the resource.
[OK!]


In [6]:
%%time
ner_model = ner_pipeline.fit(training_data)
ner_model.stages[1].write().overwrite().save('outputs/ner_wiki_glove100d_en')

2022-10-29 12:22:16.575675: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-29 12:22:16.621027: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1218] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /job:localhost/replica:0/task:0/device:CPU:0].
See below for details of this colocation group:
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=-1 requested_device_name_='/device:GPU:0' assigned_device_nam

Training started - total epochs: 500 - lr: 0.003 - batch size: 32 - labels: 8 - chars: 22 - training examples: 61
Epoch 1/500 started, lr: 0.003, dataset size: 61
Epoch 1/500 - 0.71s - loss: 21.939442 - batches: 3
Quality on validation dataset (20.0%), validation examples = 12
time to finish evaluation: 0.19s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 0	 0	 2	 0.0	 0.0	 0.0
Loc	 0	 1	 5	 0.0	 0.0	 0.0
Act	 0	 0	 14	 0.0	 0.0	 0.0
Neg	 0	 0	 1	 0.0	 0.0	 0.0
0	 22	 28	 0	 0.44	 1.0	 0.6111111
Obj	 0	 0	 5	 0.0	 0.0	 0.0
Pos	 0	 0	 2	 0.0	 0.0	 0.0
tp: 22 fp: 29 fn: 29 labels: 7
Macro-average	 prec: 0.06285714, rec: 0.14285715, f1: 0.0873016
Micro-average	 prec: 0.43137255, rec: 0.43137255, f1: 0.43137255
Epoch 2/500 started, lr: 0.0029850747, dataset size: 61
Epoch 2/500 - 0.06s - loss: 19.339823 - batches: 3
Quality on validation dataset (20.0%), validation examples = 12
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 2	 0	 0	 1.0	 1.0	 1.0
Loc	 4	 7	 1	 0.36363637	 0

time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 2	 0	 0	 1.0	 1.0	 1.0
Loc	 5	 1	 0	 0.8333333	 1.0	 0.90909094
Act	 14	 3	 0	 0.8235294	 1.0	 0.9032258
Neg	 0	 0	 1	 0.0	 0.0	 0.0
0	 22	 0	 0	 1.0	 1.0	 1.0
Obj	 4	 0	 1	 1.0	 0.8	 0.88888896
Pos	 0	 0	 2	 0.0	 0.0	 0.0
tp: 47 fp: 4 fn: 4 labels: 7
Macro-average	 prec: 0.6652661, rec: 0.6857143, f1: 0.67533547
Micro-average	 prec: 0.92156863, rec: 0.92156863, f1: 0.92156863
Epoch 15/500 started, lr: 0.0028037382, dataset size: 61
Epoch 15/500 - 0.05s - loss: 0.7595594 - batches: 3
Quality on validation dataset (20.0%), validation examples = 12
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 2	 0	 0	 1.0	 1.0	 1.0
Loc	 5	 1	 0	 0.8333333	 1.0	 0.90909094
Act	 13	 1	 1	 0.9285714	 0.9285714	 0.9285714
Neg	 0	 0	 1	 0.0	 0.0	 0.0
0	 22	 1	 0	 0.95652175	 1.0	 0.9777778
Obj	 5	 1	 0	 0.8333333	 1.0	 0.90909094
Pos	 0	 0	 2	 0.0	 0.0	 0.0
tp: 47 fp: 4 fn: 4 labels: 7
Macro-average	 prec: 0.65025

Epoch 29/500 - 0.05s - loss: 0.04381544 - batches: 3
Quality on validation dataset (20.0%), validation examples = 12
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 2	 0	 0	 1.0	 1.0	 1.0
Loc	 5	 1	 0	 0.8333333	 1.0	 0.90909094
Act	 14	 2	 0	 0.875	 1.0	 0.93333334
Neg	 0	 0	 1	 0.0	 0.0	 0.0
0	 22	 0	 0	 1.0	 1.0	 1.0
Obj	 5	 0	 0	 1.0	 1.0	 1.0
Pos	 0	 0	 2	 0.0	 0.0	 0.0
tp: 48 fp: 3 fn: 3 labels: 7
Macro-average	 prec: 0.672619, rec: 0.71428573, f1: 0.69282645
Micro-average	 prec: 0.9411765, rec: 0.9411765, f1: 0.9411765
Epoch 30/500 started, lr: 0.0026200875, dataset size: 61
Epoch 30/500 - 0.05s - loss: 0.030443594 - batches: 3
Quality on validation dataset (20.0%), validation examples = 12
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 2	 0	 0	 1.0	 1.0	 1.0
Loc	 5	 1	 0	 0.8333333	 1.0	 0.90909094
Act	 14	 2	 0	 0.875	 1.0	 0.93333334
Neg	 0	 0	 1	 0.0	 0.0	 0.0
0	 22	 0	 0	 1.0	 1.0	 1.0
Obj	 5	 0	 0	 1.0	 1.0	 1.0
Pos	 0	 0	 2	 

time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 2	 0	 0	 1.0	 1.0	 1.0
Loc	 5	 1	 0	 0.8333333	 1.0	 0.90909094
Act	 14	 0	 0	 1.0	 1.0	 1.0
Neg	 0	 0	 1	 0.0	 0.0	 0.0
0	 22	 0	 0	 1.0	 1.0	 1.0
Obj	 5	 2	 0	 0.71428573	 1.0	 0.8333334
Pos	 0	 0	 2	 0.0	 0.0	 0.0
tp: 48 fp: 3 fn: 3 labels: 7
Macro-average	 prec: 0.6496598, rec: 0.71428573, f1: 0.68044174
Micro-average	 prec: 0.9411765, rec: 0.9411765, f1: 0.9411765
Epoch 45/500 started, lr: 0.0024590164, dataset size: 61
Epoch 45/500 - 0.05s - loss: 0.037128128 - batches: 3
Quality on validation dataset (20.0%), validation examples = 12
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 2	 0	 0	 1.0	 1.0	 1.0
Loc	 5	 1	 0	 0.8333333	 1.0	 0.90909094
Act	 14	 0	 0	 1.0	 1.0	 1.0
Neg	 0	 0	 1	 0.0	 0.0	 0.0
0	 22	 0	 0	 1.0	 1.0	 1.0
Obj	 5	 2	 0	 0.71428573	 1.0	 0.8333334
Pos	 0	 0	 2	 0.0	 0.0	 0.0
tp: 48 fp: 3 fn: 3 labels: 7
Macro-average	 prec: 0.6496598, rec: 0.71428573, f1: 0.68044174
M

...

Epoch 464/500 - 0.06s - loss: 3.937006E-4 - batches: 3
Quality on validation dataset (20.0%), validation examples = 12
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 2	 0	 0	 1.0	 1.0	 1.0
Loc	 5	 0	 0	 1.0	 1.0	 1.0
Act	 14	 0	 0	 1.0	 1.0	 1.0
Neg	 0	 0	 1	 0.0	 0.0	 0.0
0	 22	 0	 0	 1.0	 1.0	 1.0
Obj	 5	 2	 0	 0.71428573	 1.0	 0.8333334
Pos	 0	 0	 2	 0.0	 0.0	 0.0
Trigger	 0	 1	 0	 0.0	 0.0	 0.0
tp: 48 fp: 3 fn: 3 labels: 8
Macro-average	 prec: 0.58928573, rec: 0.625, f1: 0.6066177
Micro-average	 prec: 0.9411765, rec: 0.9411765, f1: 0.9411765
Epoch 465/500 started, lr: 9.036145E-4, dataset size: 61
Epoch 465/500 - 0.05s - loss: 1.7606418E-4 - batches: 3
Quality on validation dataset (20.0%), validation examples = 12
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 2	 0	 0	 1.0	 1.0	 1.0
Loc	 5	 0	 0	 1.0	 1.0	 1.0
Act	 14	 0	 0	 1.0	 1.0	 1.0
Neg	 0	 0	 1	 0.0	 0.0	 0.0
0	 22	 0	 0	 1.0	 1.0	 1.0
Obj	 5	 2	 0	 0.71428573	 1.0	 0.8333334


Epoch 478/500 - 0.07s - loss: 0.0014850994 - batches: 3
Quality on validation dataset (20.0%), validation examples = 12
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 2	 0	 0	 1.0	 1.0	 1.0
Loc	 5	 0	 0	 1.0	 1.0	 1.0
Act	 14	 0	 0	 1.0	 1.0	 1.0
Neg	 0	 0	 1	 0.0	 0.0	 0.0
0	 21	 0	 1	 1.0	 0.95454544	 0.9767442
Obj	 5	 3	 0	 0.625	 1.0	 0.7692308
Pos	 0	 0	 2	 0.0	 0.0	 0.0
Trigger	 0	 1	 0	 0.0	 0.0	 0.0
tp: 47 fp: 4 fn: 4 labels: 8
Macro-average	 prec: 0.578125, rec: 0.6193182, f1: 0.598013
Micro-average	 prec: 0.92156863, rec: 0.92156863, f1: 0.92156863
Epoch 479/500 started, lr: 8.849558E-4, dataset size: 61
Epoch 479/500 - 0.07s - loss: 1.1496941E-4 - batches: 3
Quality on validation dataset (20.0%), validation examples = 12
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 2	 0	 0	 1.0	 1.0	 1.0
Loc	 5	 0	 0	 1.0	 1.0	 1.0
Act	 14	 0	 0	 1.0	 1.0	 1.0
Neg	 0	 0	 1	 0.0	 0.0	 0.0
0	 21	 0	 1	 1.0	 0.95454544	 0.9767442
Obj	 5	 3	 0	 0

Epoch 492/500 - 0.06s - loss: 2.4635793E-4 - batches: 3
Quality on validation dataset (20.0%), validation examples = 12
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 2	 0	 0	 1.0	 1.0	 1.0
Loc	 5	 0	 0	 1.0	 1.0	 1.0
Act	 14	 0	 0	 1.0	 1.0	 1.0
Neg	 0	 0	 1	 0.0	 0.0	 0.0
0	 21	 0	 1	 1.0	 0.95454544	 0.9767442
Obj	 5	 3	 0	 0.625	 1.0	 0.7692308
Pos	 0	 0	 2	 0.0	 0.0	 0.0
Trigger	 0	 1	 0	 0.0	 0.0	 0.0
tp: 47 fp: 4 fn: 4 labels: 8
Macro-average	 prec: 0.578125, rec: 0.6193182, f1: 0.598013
Micro-average	 prec: 0.92156863, rec: 0.92156863, f1: 0.92156863
Epoch 493/500 started, lr: 8.67052E-4, dataset size: 61
Epoch 493/500 - 0.06s - loss: 2.0689369E-4 - batches: 3
Quality on validation dataset (20.0%), validation examples = 12
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 2	 0	 0	 1.0	 1.0	 1.0
Loc	 5	 0	 0	 1.0	 1.0	 1.0
Act	 14	 0	 0	 1.0	 1.0	 1.0
Neg	 0	 0	 1	 0.0	 0.0	 0.0
0	 21	 0	 1	 1.0	 0.95454544	 0.9767442
Obj	 5	 3	 0	 0.

2022-10-29 12:22:49.628650: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1218] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /job:localhost/replica:0/task:0/device:CPU:0].
See below for details of this colocation group:
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=-1 requested_device_name_='/device:GPU:0' assigned_device_name_='' resource_device_name_='/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
AssignSub: CPU 
AddV2: CPU 
ScatterAdd: CPU 
NoOp: CPU 
Sqrt: CPU 
UnsortedSegmentSum: CPU 
RealDiv: CPU 
Const: CPU 
StridedSlice: CPU 
RandomUniform: CPU 
Unique: CPU 
Mul: CPU 
Add: CPU 
VariableV2: CPU 
Assign: CPU 
Identity: CPU 
Shape: CPU 
Cast: CPU 
Sub: CPU 
Gat

CPU times: user 483 ms, sys: 122 ms, total: 605 ms
Wall time: 35.6 s


In [7]:

#predictions = ner_model.transform(testing_data) 
#predictions.select(F.explode(F.arrays_zip('token.result','label.result','ner.result')).alias("cols")) \
#           .select(F.expr("cols['0']").alias("token"),
#                   F.expr("cols['1']").alias("ground_truth"),
#                   F.expr("cols['2']").alias("prediction")).show(truncate=False)

In [8]:
from sklearn.metrics import classification_report
 
#preds_df = predictions.select(F.explode(F.arrays_zip('token.result','label.result','ner.result')).alias("cols")) \
#                      .select(F.expr("cols['0']").alias("token"),
#                              F.expr("cols['1']").alias("ground_truth"),
#                              F.expr("cols['2']").alias("prediction")).toPandas()
 
#print(classification_report(preds_df['ground_truth'], preds_df['prediction']))

In [9]:
#import shutil

#shutil.make_archive("/home/sjhjrok/Documents/NLP/outputs/ner_wiki_glove100d_en", 'zip', "/home/sjhjrok/Documents/NLP/outputs/ner_wiki_glove100d_en")

In [10]:
text_list = [
    """Navagate to kitchen"""
]


In [11]:
loaded_ner_model = NerDLModel.load("outputs/ner_wiki_glove100d_en")\
   .setInputCols(["sentence", "token", "embeddings"])\
   .setOutputCol("ner")

load_ner_pipeline = Pipeline(stages=[
      document_assembler,
      tokenizer,
      embeddings,
      loaded_ner_model,
      ner_converter
 ])


2022-10-29 12:22:50.794717: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1218] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /job:localhost/replica:0/task:0/device:CPU:0].
See below for details of this colocation group:
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=-1 requested_device_name_='/device:GPU:0' assigned_device_name_='' resource_device_name_='/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
AssignSub: CPU 
AddV2: CPU 
ScatterAdd: CPU 
NoOp: CPU 
Sqrt: CPU 
UnsortedSegmentSum: CPU 
RealDiv: CPU 
Const: CPU 
StridedSlice: CPU 
RandomUniform: CPU 
Unique: CPU 
Mul: CPU 
Add: CPU 
VariableV2: CPU 
Assign: CPU 
Identity: CPU 
Shape: CPU 
Cast: CPU 
Sub: CPU 
Gat

In [12]:
from sparknlp.base import LightPipeline

text = "navigate to kitchen"
prediction_data = spark.createDataFrame([[text]]).toDF("text")
prediction_model = load_ner_pipeline.fit(prediction_data)
light = LightPipeline(prediction_model)
results = light.annotate(text)
print(results)
print(results['ner'])

{'document': ['navigate to kitchen'], 'ner_chunk': ['navigate to kitchen'], 'token': ['navigate', 'to', 'kitchen'], 'ner': ['Act', '0', 'Loc'], 'embeddings': ['navigate', 'to', 'kitchen']}
['Act', '0', 'Loc']


In [13]:
light.annotate("Navigate to kitchen")


{'document': ['Navigate to kitchen'],
 'ner_chunk': ['Navigate to kitchen'],
 'token': ['Navigate', 'to', 'kitchen'],
 'ner': ['Act', '0', 'Loc'],
 'embeddings': ['Navigate', 'to', 'kitchen']}

In [ ]:
text = input("Enter Testing Text\n")
while(text != "exit"):
    results = light.annotate(text)
    print(results['ner'])
    text = input("Enter New Text\n")

Enter Testing Text
hello start camera
['Trigger', 'Act', 'Obj']
Enter New Text
kiss 
['Obj']
Enter New Text
kiss Gwen
['Act', 'Obj']
Enter New Text
do
['Act']
Enter New Text
want
['Rqt']
Enter New Text
where
['Rqt']
Enter New Text
is
['0']
Enter New Text
smoke weed
['Act', 'Obj']


In [ ]:
#empty_df = spark.createDataFrame([['']]).toDF('text')
#pipeline_model = ner_pipeline.fit(empty_df)
#df = spark.createDataFrame(pd.DataFrame({'text': text_list}))
#result = pipeline_model.transform(df)

In [ ]:




#from sparknlp_display import NerVisualizer



#result = predictions

#NerVisualizer().display(
#    result = result.collect()[0],
#    label_col = 'ner_chunk',
#    document_col = 'document'
#)




